In [11]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters
import requests 
import json
import pandas as pd
from bs4 import BeautifulSoup
import logging
import emoji
import schedule
import time

###     JSON file example    ###
#
# {
#   "1": [
#     {
#       "stock_number": "005930",
#       "stock_name": "삼성전자",
#       "stock_price": "57500"
#     }
#   ],
#   "2": [
#     {
#       "stock_number": "000660",
#       "stock_name": "LG전자",
#       "stock_price": "85600"
#     }
#   ]
# }
#
###     JSON file example    ###

# JSON data read
with open('./stock_data.json' ,encoding = "UTF-8") as file:
    json_data = json.load(file)
    temp = pd.DataFrame(json_data)
    df = temp.T
    
# Active Bot with the token
token = "1461066934:AAHQLhV3b7PJ-EJX-JNJdYv0tz2fZBnWmhs"
bot = telegram.Bot(token)

# updater 
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

# Pre-requirement Data for send_stock_recommendation
url = "https://finance.naver.com/item/main.nhn?code=" 
ment1 = "<<  끝투's 오늘의 추천 종목  >>\n\n"
ment = ment1
image = 'test_image.jpg'

# help page
help_text = "<<  끝투 사용방법  >>\n\n1.  /help : 도움말 페이지 \n\n2.  /recommend : 오늘의 추천 주식\n\n3.  buy : 주식구매\n"
greeting_text = "끝투에 오신 것을 환영합니다!\n"

# Getting Stock Information
def get_stock_price(code):
    result = requests.get(url+code)
    bs_obj = BeautifulSoup(result.content,"html.parser")
    no_today = bs_obj.find("p", {"class": "no_today"})
    blind = no_today.find("span", {"class": "blind"}) # 태그 span, 속성값 blind 찾기
    now_price = blind.text
    return now_price

# make stock name as same length with each other
def equalizer(a):
    gong = ""
    for i in range (9-a):
        gong = gong + "  "
    return gong    

# make 'ment' for sending telegram
def send_stock_recommendation(df):
    url = "https://finance.naver.com/item/main.nhn?code=" 
    ment1 = "<<  끝투's 오늘의 추천 종목  >>\n\n"
    ment = ment1
    image = 'test_image.jpg'    
    for idx,row in df.iterrows():
        info = row
        code = info['stock_number']
        code_name = info['stock_name']
        name_len_a = len(code_name)
        price = info['stock_price']
        price = str(price)
        price = price.rjust(8)
        url_with_code = url + code
        updated_url = "[" + code_name + "]" + "(" + url_with_code + ")"
        temp = idx + '. ' + updated_url + equalizer(name_len_a) +'가격'+price + '    매수 클릭\n\n'   ####you can use get_stock_price(code) for the price
        ment = ment + temp
    # send message to users     
    bot.sendMessage(chat_id = 1437875774,text = ment, parse_mode = 'Markdown',disable_web_page_preview=True)

# Defalult Greeting    
bot.send_photo(chat_id = 1437875774, photo=open(image, 'rb'))
bot.sendMessage(chat_id = 1437875774, text= greeting_text)
bot.sendMessage(chat_id = 1437875774, text= help_text)
    
# help page    
def help(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text= help_text)

start_handler = CommandHandler('help', help)
dispatcher.add_handler(start_handler)

def recommend(update, context):
    send_stock_recommendation(df)

start_handler = CommandHandler('recommend', recommend)
dispatcher.add_handler(start_handler)

# start
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)


# start polling
updater.start_polling(timeout=3, clean=True)
updater.idle()



